In [72]:
import numpy as np
import pandas as pd

In [73]:
data = pd.read_csv('../data/juliet.csv.zip')

In [74]:
small_data = data.iloc[:10,:]

In [75]:
small_data.iloc[0]

Unnamed: 0                                                     0
testcase_ID                                                61940
filename       000/061/940/CWE114_Process_Control__w32_char_c...
code           /* TEMPLATE GENERATED TESTCASE FILE\nFilename:...
flaw                                                     CWE-114
flaw_loc                                                     121
CWE-015                                                    False
CWE-023                                                    False
CWE-036                                                    False
CWE-078                                                    False
CWE-090                                                    False
CWE-114                                                     True
CWE-121                                                    False
CWE-122                                                    False
CWE-123                                                    False
CWE-124                  

In [76]:
import clang.cindex
import os
clang.cindex.Config.set_library_file('/lib/x86_64-linux-gnu/libclang-8.so.1')

Exception: library file must be set before before using any other functionalities in libclang.

In [77]:
index_clang = clang.cindex.Index.create()
translation_unit = []
for i in range(10):
    value = small_data.iloc[i]
    translation_unit.append(index_clang.parse(path=value.filename, unsaved_files=[(value.filename, value.code)]))

In [78]:
translation_unit

In [79]:
root = []
for i in range(10):
    root.append(translation_unit[i].cursor)

In [80]:
import snap

In [88]:
print(root[0])

In [89]:
def print_ast(cursor, deep=0):
    print(' '.join((deep*'    ', str(cursor.kind), str(cursor.spelling))))
    for child in cursor.get_children():
        print_ast(child, deep+1)

In [95]:
def print_ast(node, deep=0):
    print(' '.join((deep*'    ', str(node.kind), str(node.identifier))))
    for child in node.get_children():
        print_ast(child, deep+1)

In [96]:
print_ast(root[1])

 CursorKind.TRANSLATION_UNIT 2539


AttributeError: 'Cursor' object has no attribute 'identifier'

In [81]:
identifier = 1

def number_each_node(node):
    global identifier
    
    node.identifier = identifier
    identifier += 1
    
    node.children = list(node.get_children())
    for child in node.children:
        number_each_node(child)

In [92]:
map(number_each_node, root)

In [83]:
tree_all = [snap.TNGraph.New()]*10
                
def all_tree_edge(root, index):
    node = root[index]
    tree = tree_all[index]
    
    def tree2edgelist(node):
        tree.AddNode(node.identifier)

        for child in node.children:
            tree2edgelist(child)
            tree.AddEdge(node.identifier, child.identifier)

    tree_all[index] = tree

In [84]:
for i in range(10):
    all_tree_edge(root, i)

In [85]:
tree_all

[<snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >,
 <snap.PNGraph; proxy of <Swig Object of type 'PNGraph *' at 0x7f698a668fc0> >]

In [86]:
for i in range(10):
    snap.SaveEdgeList(tree_all[0], '../data/mygraphall' + str(i) + '.txt')